[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/DM871/dm871.github.io/blob/main/notebooks/factory_planning_maintenance_sol.ipynb)

# Factory Planning and Machine Maintenance

A firm makes seven products $1,\ldots,7$ on the following machines: 4
grinders, 2 vertical drills, 3 horizontal drills, 1 borer, and 1 planer.

Each product yields a certain contribution to the profit (defined as
selling price minus cost of raw materials expressed in Euro/unit). These
quantities (in Euro/unit) together with the production times
(hours/unit) required on each process are given below.

     product     1      2      3      4      5      6     7
     profit      10     6      8      4      11     9     3
     grinding    0.5    0.7    0      0      0.3    0.2   0.5
     vdrill      0.1    0.2    0      0.3    0      0.6   0
     hdrill      0.2    0      0.8    0      0      0     0.6
     boring      0.05   0.03   0      0.07   0.1    0     0.08
     planning    0      0      0.01   0      0.05   0     0.05


In the first month (January) and the five subsequent months certain
machines will be down for maintenance. These machines will be:

     January   1 grinder
     February  2 hdrill
     March     1 borer
     April     1 vdrill
     May       1 grinder
     May       1 vdrill
     June      1 planer
     June      1 hdrill


There are marketing limitations on each product in each month. That is,
in each month the amount sold for each product cannot exceed these
values:

     product    1     2      3     4     5      6     7
     January    500   1000   300   300   800    200   100
     February   600   500    200   0     400    300   150
     March      300   600    0     0     500    400   100
     April      200   300    400   500   200    0     100
     May        0     100    500   100   1000   300   0
     June       500   500    100   300   1100   500   60


It is possible to store products in a warehouse. The capacity of the
storage is 100 units per product type per month. The cost is 0.5 Euro
per unit of product per months. There are no stocks in the first month
but it is desired to have a stock of 50 of each product type at the end
of June.

The factory works 6 days a week with two shifts of 8 hours each day. (It
can be assumed that each month consists of 24 working days.)

The factory wants to determine a production plan, that is, the quantity
to produce, sell and store in each month for each product, that
maximizes the total profit.



## Task 1

Model the factory planning problem for the month of January as an LP
problem.

## Task 2

Model the multi-period (from January to June) factory planning problem
as an LP problem. Use mathematical notation and indicate in general
terms how many variables and how many constraints your model has.

- How many variables and how many constraints do we have in the  model in terms of number of products and planning months?
- What is the structure of the matrix generated for the model?


## Task 3

Implement the single-period model (Task 1) in a SpreadSheet.
Implement the multi-period model (Task 2) in Python and Gurobi. Solve
the problem on the data given.

- Report and comment relevant information from the execution of the solver.
- Report the production plan, that is, how much of each product should the factory produce in the months.
- Indicate which resource capacity could be convinient to increase in some months and the impact that such increase would have on the total profit.
- Indicate which change in price could make appealing producing a product that is not in production in a certain month.

Here are the data of the problem:


In [1]:
from collections import OrderedDict

class Data:
    def __init__(self):
        self.products = [1, 2, 3, 4, 5, 6, 7];
        self.machines = ["grinder","vdrill","hdrill","borer","planer"]
        self.months = ["january","february","march","april","may","june"]

        self.profits = [10, 6, 8, 4, 11, 9, 3]

        tmp = {
        'grinder'     :[0.5,    0.7,    0,      0,      0.3,    0.2,   0.5],
        'vdrill'      :[0.1 ,   0.2 ,   0   ,   0.3 ,   0   ,   0.6,   0],
        'hdrill'      :[0.2 ,   0   ,   0.8 ,   0   ,   0   ,   0  ,   0.6],
        'borer'      :[0.05,   0.03,   0   ,   0.07,   0.1 ,   0  ,   0.08],
        'planer'    :[0   ,   0   ,   0.01,   0   ,   0.05,   0  ,   0.05]
        }
        self.coeff=OrderedDict()
        for m in self.machines:
            for (j,p) in enumerate(self.products):
                self.coeff[m,p] = tmp[m][j]

        self.capacity = {"grinder": 4,"vdrill": 2,"hdrill": 3, "borer": 1, "planer": 1}

        tmp = {
        "grinder": [("january", 1), ("may", 1)],
        "hdrill": [("february", 2),("june", 1)],
        "borer":  [("march", 1)],
        "vdrill": [("april", 1),("may", 1)],
        "planer": [("june", 1)]
        }


        self.maintainance = OrderedDict()
        for m in self.machines:
            for t in self.months:
                self.maintainance[m,t] = 0
            if m in tmp:
                for s in tmp[m]:
                    self.maintainance[m,s[0]]=s[1]

        tmp = {
        "january":    [500   ,1000   ,300   ,300   ,800    ,200   ,100],
        "february":   [600   ,500    ,200   ,0     ,400    ,300   ,150],
        "march":      [300   ,600    ,0     ,0     ,500    ,400   ,100],
        "april":      [200   ,300    ,400   ,500   ,200    ,0     ,100],
        "may":        [0     ,100    ,500   ,100   ,1000   ,300   ,0  ],
        "june":       [500   ,500    ,100   ,300   ,1100   ,500   ,60 ]
        }

        self.market_limits = OrderedDict()
        for m in self.months:
            for (j,p) in enumerate(self.products):
                self.market_limits[m,p] = tmp[m][j]

    def printData(self):
        print("Months:", self.months)
        print("Products:", self.products)
        print("Machines:", self.machines)
        print("Coefficients: ",self.coeff)
        print("Market_limits:", self.market_limits)
        print("Maitainance:", self.maintainance)

In [2]:
data = Data()
data.printData()

Months: ['january', 'february', 'march', 'april', 'may', 'june']
Products: [1, 2, 3, 4, 5, 6, 7]
Machines: ['grinder', 'vdrill', 'hdrill', 'borer', 'planer']
Coefficients:  OrderedDict([(('grinder', 1), 0.5), (('grinder', 2), 0.7), (('grinder', 3), 0), (('grinder', 4), 0), (('grinder', 5), 0.3), (('grinder', 6), 0.2), (('grinder', 7), 0.5), (('vdrill', 1), 0.1), (('vdrill', 2), 0.2), (('vdrill', 3), 0), (('vdrill', 4), 0.3), (('vdrill', 5), 0), (('vdrill', 6), 0.6), (('vdrill', 7), 0), (('hdrill', 1), 0.2), (('hdrill', 2), 0), (('hdrill', 3), 0.8), (('hdrill', 4), 0), (('hdrill', 5), 0), (('hdrill', 6), 0), (('hdrill', 7), 0.6), (('borer', 1), 0.05), (('borer', 2), 0.03), (('borer', 3), 0), (('borer', 4), 0.07), (('borer', 5), 0.1), (('borer', 6), 0), (('borer', 7), 0.08), (('planer', 1), 0), (('planer', 2), 0), (('planer', 3), 0.01), (('planer', 4), 0), (('planer', 5), 0.05), (('planer', 6), 0), (('planer', 7), 0.05)])
Market_limits: OrderedDict([(('january', 1), 500), (('january', 2)

In [3]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install gurobipy>=12 # note: use python version <=3.12.7.

zsh:1: 12 not found


In [4]:
import gurobipy as gp
from gurobipy import GRB

In [5]:
m = gp.Model("fpmm")
m.setParam(GRB.param.Method, 0)

x={}
for i in data.products:
    for (t_int, t_string) in enumerate(data.months):
        x[i,t_int]=m.addVar(lb=0.0,ub=GRB.INFINITY,obj=0.0,vtype=GRB.CONTINUOUS,name="x_%s_%s" % (i,t_int))

s={}
for i in data.products:
    for (t_int, t_string) in enumerate(data.months):
        s[i,t_int]=m.addVar(lb=0.0,ub=GRB.INFINITY,obj=0.0,vtype=GRB.CONTINUOUS,name="s_%s_%s" % (i,t_int))

h={}
for i in data.products:
    for (t_int, t_string) in enumerate(data.months):
        h[i,t_int]=m.addVar(lb=0.0,ub=100,obj=0.0,vtype=GRB.CONTINUOUS,name="h_%s_%s" % (i,t_int))


m.setObjective(gp.quicksum(data.profits[i0]*s[i1,t_int]-0.5*h[i1,t_int]
                        for (i0,i1) in enumerate(data.products)
                        for (t_int,t_string) in enumerate(data.months)),
                GRB.MAXIMIZE)

# machine capacities
c={}
for j in data.machines:
    for (t_int, t_string) in enumerate(data.months):
        c[j,t_string]=m.addConstr(gp.quicksum(data.coeff[j,i]*x[i,t_int] for i in data.products) <= 384*(data.capacity[j]-data.maintainance[j,t_string]),"cap_%s" % j)

# mass balance
for i in data.products:
    for (t_int, t_string) in enumerate(data.months):
        if t_int==0:
            m.addConstr(x[i,t_int]==s[i,t_int]+h[i,t_int],"bal0_%s_%s" % (i,t_int))
        else:
            m.addConstr(h[i,t_int-1]+x[i,t_int]==s[i,t_int]+h[i,t_int],"bal_%s_%s" % (i,t_int))

for i in data.products:
    for (t_int, t_string) in enumerate(data.months):
        m.addConstr(s[i,t_int]<=data.market_limits[t_string, i],"market_limits_%s_%s" % (i,t_int) )

for i in data.products:
    m.addConstr(h[i,5]>=50)

m.write("model.lp")
m.optimize()


Set parameter Username
Set parameter LicenseID to value 2599106
Academic license - for non-commercial use only - expires 2025-12-13
Set parameter Method to value 0
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[arm])

CPU model: Apple M1 Max
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 121 rows, 126 columns and 330 nonzeros
Model fingerprint: 0xace1d00f
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [5e-01, 1e+01]
  Bounds range     [1e+02, 1e+02]
  RHS range        [5e+01, 2e+03]
Presolve removed 116 rows and 110 columns
Presolve time: 0.00s
Presolved: 5 rows, 16 columns, 21 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.0175000e+04   0.000000e+00   3.340000e+02      0s
Extra simplex iterations after uncrush: 2
      12    9.3715179e+04   0.000000e+00   0.000000e+00      0s

Solved in 12 iterations and 0.00 seconds (0.00 work units)
Optimal objective

In [6]:
def printSolution():
    print("x[i,t]=")
    sss="\t"
    for (t_int, t_string) in enumerate(data.months):
        sss+="{0:>{width}}".format(t_string,width=13)
    print(sss)
    for i in data.products:
        sss=str(i)+"\t"
        for (t_int, t_string) in enumerate(data.months):
            sss+="{0:>{width}}".format(str(x[i,t_int].x),width=13)
        print(sss)

    print("\ns[i,t]=")
    sss="\t"
    for (t_int, t_string) in enumerate(data.months):
        sss+="{0:>{width}}".format(t_string,width=13)
    print(sss)
    for i in data.products:
        sss=str(i)+"\t"
        for (t_int, t_string) in enumerate(data.months):
            sss+="{0:>{width}}".format(str(s[i,t_int].x),width=13)
        print(sss)

    print("\nh[i,t]=")
    sss="\t"
    for (t_int, t_string) in enumerate(data.months):
        sss+="{0:>{width}}".format(t_string,width=13)
    print(sss)
    for i in data.products:
        sss=str(i)+"\t"
        for (t_int, t_string) in enumerate(data.months):
            sss+="{0:>{width}}".format(str(h[i,t_int].x),width=13)
        print(sss)

    print("\ns[i,t].rc= (reduced costs)")
    sss="\t"
    for (t_int, t_string) in enumerate(data.months):
        sss+="{0:>{width}}".format(t_string,width=13)
    print(sss)
    for i in data.products:
        sss=str(i)+"\t"
        for (t_int, t_string) in enumerate(data.months):
            sss+="{0:>{width}}".format(str(s[i,t_int].rc),width=13)
        print(sss)

    print("\nc[i,t]= (marginal values)")
    sss="\t"
    for (t_int, t_string) in enumerate(data.months):
        sss+="{0:>{width}}".format(t_string,width=13)
    print(sss)
    for j in data.machines:
        sss=str(j)+"\t"
        for (t_int, t_string) in enumerate(data.months):
            sss+="{0:>{width}}".format(str(c[j,t_string].pi),width=13)
        print(sss)

printSolution()

x[i,t]=
	      january     february        march        april          may         june
1	        500.0        700.0          0.0        200.0          0.0        550.0
2	888.5714285714287        600.0          0.0        300.0        100.0        550.0
3	        382.5        117.5          0.0        400.0        600.0          0.0
4	        300.0          0.0          0.0        500.0        100.0        350.0
5	        800.0        500.0          0.0        200.0       1100.0          0.0
6	        200.0        300.0        400.0          0.0        300.0        550.0
7	          0.0        250.0          0.0        100.0        100.0          0.0

s[i,t]=
	      january     february        march        april          may         june
1	        500.0        600.0        100.0        200.0          0.0        500.0
2	888.5714285714286        500.0        100.0        300.0        100.0        500.0
3	        300.0        200.0          0.0        400.0        500.0         50.0
4	   

## Task 4

Here, instead of stipulating when each machine is down for maintenance, it is
desired to find the best month for each machine to be down.

Each machine must be down for maintenance in one month of the six
apart from the grinding machines, only two of which need to be down in any
six months.

Extend the model that correctly addressed tasks 2 and 3 to allow it to make these extra decisions.

- How many variables did you need to add? What is the domain of these
  variables?

- Has the matrix of the problem a similar structure to the one
  of the point above?

- Is the solution from Task 3 a valid solution to this problem?
  What information can it bear in this new case?

- Implement and solve the model in Python and Gurobi. After how many
  nodes in the branch and bound tree is the optimal solution found? And
  after how many is it proven optimal?

- How much worth is the extra flexibility of choosing when to place
  downtimes?

- The column ``LP iter'' indicates the number of simplex iterations for solving the LP problem. Why is this number increasing through the search?

- Can you devise another model and compare them computationally?  Which is best? Why?